In [1]:
import os
import sys
# sys.path.append("../meta_infomax/")
sys.path.append("..")

from typing import Iterator, List, Dict, Union
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

import pandas as pd


from sklearn.model_selection import train_test_split

torch.manual_seed(1)
%load_ext autoreload
%autoreload 2

from pathlib import Path
from meta_infomax.datasets import utils
from meta_infomax.datasets.fudan_reviews import MultiTaskDataset

# import torchtext
# from torchtext.data import Field, LabelField, TabularDataset, Dataset
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler, BatchSampler

from meta_infomax.models.sentiment_classifier import SentimentClassifier
from meta_infomax.models.feed_forward import FeedForward
import logging

In [5]:
utils.init_logging()

In [2]:
bert, tokenizer, embedding_dim = utils.get_transformer('bert-base-uncased')

In [16]:
# %debug
mtl = MultiTaskDataset(tokenizer=tokenizer, data_dir='../meta_infomax/data/mtl-dataset/', split='train')

100%|██████████| 17742/17742 [00:42<00:00, 421.58it/s]


In [18]:
mtl

In [7]:
domains = ['apparel', 'video']

In [14]:
mtl.data.loc[mtl.data['domain'].isin(domains), :].reset_index(drop=True)

,label,text,domain,tokenized
0,0,really uncomfortable . i wore them twice and t...,apparel,"[2428, 8796, 1012, 1045, 5078, 2068, 3807, 199..."
1,0,sometimes you get a good bargain - cheap price...,apparel,"[2823, 2017, 2131, 1037, 2204, 17113, 1011, 10..."
2,1,this is very comfy and pretty and a great pric...,apparel,"[2023, 2003, 2200, 4012, 12031, 1998, 3492, 19..."
3,1,these keep your feet warm and toastie . will w...,apparel,"[2122, 2562, 2115, 2519, 4010, 1998, 15174, 26..."
4,1,these boots fit like a glove and are so sleek ...,apparel,"[2122, 6879, 4906, 2066, 1037, 15913, 1998, 20..."
...,...,...,...,...
2555,0,why should one buy this series ? what about th...,video,"[2339, 2323, 2028, 4965, 2023, 2186, 1029, 205..."
2556,1,"many people , when they think of buddhism , th...",video,"[2116, 2111, 1010, 2043, 2027, 2228, 1997, 113..."
2557,0,i enjoyed this movie . i orginially saw this m...,video,"[1045, 5632, 2023, 3185, 1012, 1045, 8917, 233..."
2558,1,celebrated scholar lord clark invites viewers ...,video,"[6334, 6288, 2935, 5215, 18675, 7193, 2000, 36..."


In [24]:
# batch = next(iter(DataLoader(mtl, batch_size=10, collate_fn=mtl.collate, shuffle=True)))

len(DataLoader(mtl, batch_size=10, collate_fn=mtl.collator, shuffle=True)) * 4

7100

In [27]:
len(mtl.get_domain('apparel'))

1280

In [13]:
x, masks, labels, domains = batch['x'], batch['masks'], batch['labels'], batch['domains']

In [14]:
domains

['music',
 'electronics',
 'video',
 'camera_photo',
 'books',
 'toys_games',
 'software',
 'sports_outdoors',
 'books',
 'music']

In [41]:
with torch.no_grad():
    outputs = bert(input_ids=x, attention_mask=masks)

In [45]:
encoding = outputs[0]

In [47]:
encoding.shape

torch.Size([10, 322, 768])

In [44]:
bert.forward??

In [33]:
masks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [37]:
x[3]

tensor([  101,  2023,  2003,  1996,  5409,  2338,  1997,  8101,  6256,  1045,
         2031,  2412,  3191,  1012, 13426,  3084,  2053,  3535,  2000,  4957,
         2028, 20423,  2000,  1996,  2279,  1012,  2009,  9631,  2066,  2382,
         2086,  1997,  8835,  3964, 17835,  2362,  5292, 21890, 26154,  2135,
         1010,  4375,  2000,  1037,  4619,  3076,  1010,  1998,  3133,  2046,
         1037,  3274,  2058,  1037,  5353,  1012,  2002, 24528, 29201,  2015,
         2370,  2195,  2335,  1012,  1045,  6955,  2026,  6100,  2000,  1996,
         3075,  2004,  2574,  2004,  1045,  2001,  2589,  3752,  2009,  1010,
         2061,  1045,  2064,  2025, 21893,  1996,  5530,  2073,  1996, 26917,
         2015,  5258,  1025,  2021,  2195,  2335,  1045,  9131,  3752,  1037,
         6019,  2073, 13426,  2018,  2074,  2056,  1996,  6635,  4500,  1037,
         2261,  5530,  3041,  1012,  2000,  2191,  5609,  4788,  1010,  2002,
         2130,  2758,  1996, 22759, 24528, 29201,  2015,  2370, 

In [156]:
labels

[1, 1, 1, 1, 0, 0, 1, 1, 0, 0]

In [136]:
domains

['health_personal_care',
 'health_personal_care',
 'electronics',
 'apparel',
 'video',
 'books',
 'baby',
 'video',
 'video',
 'books']

In [161]:
apparel = mtl.get_domain('apparel')

In [162]:
apparel[0]

{'label': 0,
 'text': 'really uncomfortable . i wore them twice and then threw them out . they sagged like a sad pair of tights you used to wear when you were little . and they became transparent like they were going to rip at any moment . not worth the $ 5 i paid',
 'domain': 'apparel',
 'tokenized': [2428,
  8796,
  1012,
  1045,
  5078,
  2068,
  3807,
  1998,
  2059,
  4711,
  2068,
  2041,
  1012,
  2027,
  26719,
  2066,
  1037,
  6517,
  3940,
  1997,
  4389,
  2015,
  2017,
  2109,
  2000,
  4929,
  2043,
  2017,
  2020,
  2210,
  1012,
  1998,
  2027,
  2150,
  13338,
  2066,
  2027,
  2020,
  2183,
  2000,
  10973,
  2012,
  2151,
  2617,
  1012,
  2025,
  4276,
  1996,
  1002,
  1019,
  1045,
  3825]}

In [163]:
domain_dataloaders = mtl.domain_dataloaders(['apparel', 'electronics', 'video'],
                       batch_size=10, shuffle=True, collate_fn=multitask_collate)

In [164]:
next(iter(domain_dataloaders['apparel']))

{'x': tensor([[ 101, 2122, 2020,  ...,    0,    0,    0],
         [ 101, 2023, 4031,  ...,    0,    0,    0],
         [ 101, 1045, 4149,  ...,    0,    0,    0],
         ...,
         [ 101, 2122, 2024,  ...,    0,    0,    0],
         [ 101, 1045, 2228,  ...,    0,    0,    0],
         [ 101, 1996, 7094,  ...,    0,    0,    0]]),
 'masks': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': [0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
 'domains': ['apparel',
  'apparel',
  'apparel',
  'apparel',
  'apparel',
  'apparel',
  'apparel',
  'apparel',
  'apparel',
  'apparel']}

In [72]:
texts = list(mtl.data[mtl.data['domain'] == 'imdb'].text)
longest = sorted(texts, key=lambda x: len(x), reverse=True)[:20]
[len(x.split()) for x in longest]

[1150,
 1120,
 1108,
 1080,
 1122,
 1050,
 1070,
 1068,
 978,
 1038,
 1059,
 987,
 901,
 935,
 930,
 960,
 940,
 946,
 910,
 923]

In [3]:
# data = datasets.fudan_reviews.get_fudan_datasets(tokenizer, datasets=['apparel'], data_dir='../meta_infomax/tests/fixtures/')
data = fudan_reviews.get_fudan_datasets(tokenizer) 

processing dataset: apparel


Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1308 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (814 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

processing dataset: baby


Token indices sequence length is longer than the specified maximum sequence length for this model (752 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (583 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (641 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (512 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

processing dataset: books


...

Token indices sequence length is longer than the specified maximum sequence length for this model (569 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (568 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

processing dataset: toys_games


Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (797 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (990 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

processing dataset: video


Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (675 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [4]:
thrown_away = utils.remove_outlier_lengths(data) # call only once!

In [5]:
domains = ['apparel', 'baby', 'books', 'camera_photo', 'electronics',
            'health_personal_care', 'imdb', 'kitchen_housewares', 'magazines',
            'music', 'software', 'sports_outdoors', 'toys_games', 'video']
for domain in domains:
    print(len(data[domain]['train']), len(data[domain]['val']), len(data[domain]['test']))

1272 318 399
1193 298 399
1272 318 399
1268 318 399
1270 318 399
1272 318 399
1272 318 399
1272 318 399
1248 312 399
1272 318 399
1204 301 399
1272 318 399
1272 318 399
1272 318 399


In [14]:
ff = FeedForward(768, 3, [512, 256, 2], nn.ReLU())
feedforward_optimizer = optim.Adam(ff.parameters())
bert_optimizer = optim.AdamW(bert.parameters(), lr=2e-5)

model = SentimentClassifier(bert, ff)

In [7]:
model.head

FeedForward(
  (_linear_layers): ModuleList(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=2, bias=True)
  )
  (_dropout): ModuleList(
    (0): Dropout(p=0.0, inplace=False)
    (1): Dropout(p=0.0, inplace=False)
    (2): Dropout(p=0.0, inplace=False)
  )
)

In [8]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

In [12]:
# one classifier for each domain
feedforward_classifiers = {domain: FeedForward(768, 3, [512, 256, 2], nn.ReLU()) for domain in data.keys()}
feedforward_optimizers = {domain: optim.Adam(net.parameters()) for domain, net in feedforward_classifiers.items()}

# needs to be adam warmup variant
bert_optimizer = optim.AdamW(bert.parameters(), lr=2e-5)

models = {domain: SentimentClassifier(bert, ff) for domain, ff in feedforward_classifiers.items()}

In [13]:
domains_batch = utils.sample_domains(data)

In [9]:
train_domains = ['apparel', 'baby', 'books', 'camera_photo', 'health_personal_care',
                 'imdb', 'kitchen_housewares', 'magazines', 'sports_outdoors', 'toys_games'] 
valid_domains = ['software', 'electronics']
test_domains = ['music', 'video']

In [10]:
train_iter = utils.get_iterators(data, include_domains=train_domains, split='train', batch_size=8, collapse_domains=True)
valid_iter = utils.get_iterators(data, include_domains=valid_domains, split='val', batch_size=16, collapse_domains=True)
test_iter = utils.get_iterators(data, include_domains=test_domains, split='test', batch_size=16, collapse_domains=True)

In [11]:
batch = next(iter(train_iter['all']))
batch


[torchtext.data.batch.Batch of size 8]
	[.label]:[torch.LongTensor of size 8]
	[.text]:('[torch.LongTensor of size 8x459]', '[torch.LongTensor of size 8]')

In [12]:
for batch in train_iter['all']:
    (text, text_lengths), label = batch.text, batch.label
    if text.shape[1] <= 512:
        break
model(text, label), label

({'logits': tensor([[ 0.0274,  0.0002],
          [ 0.0285, -0.0025],
          [ 0.0181, -0.0026],
          [ 0.0144, -0.0011],
          [ 0.0003, -0.0091],
          [ 0.0329, -0.0005],
          [ 0.0375, -0.0122],
          [ 0.0250, -0.0303]], grad_fn=<AddmmBackward>),
  'loss': tensor(0.6835, grad_fn=<NllLossBackward>),
  'acc': 0.75},
 tensor([1, 0, 0, 1, 0, 0, 0, 0]))

In [15]:
losses = []
for i in range(100):
    output = model(text, label)
    logits = output['logits']
    loss = output['loss']
    bert_optimizer.zero_grad()
    feedforward_optimizer.zero_grad()
    loss.backward()
    bert_optimizer.step()
    feedforward_optimizer.step()
    print(loss.item())
    losses.append(loss.item())
    

0.7461268901824951
0.5762065052986145
0.5211427807807922


KeyboardInterrupt: 